This is exploring genai sdk based on https://googleapis.github.io/python-genai/

In [1]:
import os
from dotenv import load_dotenv

from google import genai
from google.genai import types

load_dotenv()

True

In [2]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [8]:
response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
print(response.text)

The sky is blue because of a phenomenon called **Rayleigh scattering**. Here's a breakdown:

*   **Sunlight is made of all colors:** Sunlight appears white, but it's actually composed of all the colors of the rainbow (red, orange, yellow, green, blue, indigo, violet).

*   **Sunlight enters the atmosphere:** When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).

*   **Scattering of light:** This collision causes the light to scatter in different directions.

*   **Rayleigh Scattering:** Rayleigh scattering is a type of scattering that occurs when the size of the particles (air molecules) is much smaller than the wavelength of the light.  The amount of scattering is inversely proportional to the fourth power of the wavelength. This means that shorter wavelengths (blue and violet) are scattered much more strongly than longer wavelengths (red and orange).

*   **Why blue, not violet?**  Violet light is scattered even more than blue 

# Tool call

In [12]:

def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return 'sunny'


response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='What is the weather like in Boston?',
    config=types.GenerateContentConfig(
        tools=[get_current_weather],
        
    ),
)

# python functions get executed automatically
print(response.text)

The weather in Boston is sunny.



In [58]:
weather_function = types.FunctionDeclaration(
    name='get_current_weather',
    description='Get the current weather in a given location',
    parameters=types.Schema(
        type='OBJECT',
        properties={
            'location': types.Schema(
                type='STRING',
                description='The city and state, e.g. San Francisco, CA',
            ),
        },
        required=['location'],
    ),
)

city_function = types.FunctionDeclaration(
    name='get_city_name',
    description='Get the city name from city description and features',
    parameters=types.Schema(
        type='OBJECT',
        properties={
            'description': types.Schema(
                type='STRING',
                description='The city description, e.g. The city is known for its history and culture.',
            ),
        },
        required=['description'],
    ),
)

tool = types.Tool(function_declarations=[city_function, weather_function])

response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='Weather of delhi and name of the city with taj mahal?',
    config=types.GenerateContentConfig(
        tools=[tool],
    ),
)
print(response.text)
print(response.function_calls)

None
[FunctionCall(id=None, args={'location': 'Delhi, India'}, name='get_current_weather'), FunctionCall(id=None, args={'description': 'The city with Taj Mahal'}, name='get_city_name')]


In [59]:


city_function = types.FunctionDeclaration(
    name='get_city_name',
    description='Get the city name from city description and features',
    parameters=types.Schema(
        type='OBJECT',
        properties={'description': 
                    {'title': 'The city description, e.g. The city is known for its history and culture.', 'type': 'string'}
                    }
    ),
)

tool = types.Tool(function_declarations=[city_function])

response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='name of the city with multiple windows?',
    config=types.GenerateContentConfig(
        tools=[tool],
    ),
)
print(response.text)
print(response.function_calls)

None
[FunctionCall(id=None, args={'description': 'city with multiple windows'}, name='get_city_name')]


# Chat (session history retained)

In [9]:
chat = client.chats.create(model='gemini-2.0-flash-001')
response = chat.send_message('What is the best selling novel of all times',
                             config = types.GenerateContentConfig(system_instruction="You are a cow")
)
print(response.text)

Moo! Well now, that's a question that's been chewed on more than a cud of good clover! It's a tricky one, see, 'cause different folks count things different ways. But if we're talkin' about the most copies *sold*, the Bible usually takes the cake... or maybe the whole pasture! It's been translated into so many languages and spread far and wide, it's hard to keep track. 

But if you're thinkin' about a "novel" in the sense of a fictional story, some folks argue it's "Don Quixote" by Miguel de Cervantes. It's mighty old and been read by generations.

So, depending on what you're lookin' for, the answer might be different! Now, if you'll excuse me, I've got some grass to munch on. Moo!



In [51]:
response = chat.send_message('What is the story')
print(response.text)

"Don Quixote," originally titled "The Ingenious Gentleman Don Quixote of La Mancha," tells the story of Alonso Quixano, a middle-aged gentleman from La Mancha, Spain, who is consumed by reading chivalric romances. He becomes so obsessed that he loses his sanity and decides to become a knight-errant himself, renaming himself Don Quixote.

Here's a breakdown of the core elements of the story:

*   **The Delusion:** Don Quixote believes everything he's read in the romances is true. He sees himself as a noble knight on a quest for justice and adventure. He's completely out of touch with reality and interprets ordinary things as fantastical adventures.

*   **Sancho Panza:** Don Quixote recruits a simple, illiterate farmer named Sancho Panza as his squire. Sancho is the voice of reason and practicality, constantly trying (and failing) to bring Don Quixote back to earth. He's motivated by the promise of being made governor of an island. Sancho provides comic relief and a grounding perspectiv

# Chat with tools

In [92]:
chat = client.chats.create(model='gemini-2.0-flash-001')
response = chat.send_message('name of the city with multiple windows?')
print(response.text)

This is a riddle! The answer is **Window Rock**.

Window Rock is the capital of the Navajo Nation, and the name refers to a natural sandstone arch with a large hole in it, which could be described as a "window."



In [93]:
chat = client.chats.create(model='gemini-2.0-flash-001')
response = chat.send_message(
    'name of the city with multiple windows?',
    config = types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=[city_function])],
    )
)
print(response.text)

print(response.function_calls)

None
[FunctionCall(id=None, args={'description': 'city with multiple windows'}, name='get_city_name')]


In [94]:
chat.get_history()

[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='name of the city with multiple windows?')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'description': 'city with multiple windows'}, name='get_city_name'), function_response=None, inline_data=None, text=None)], role='model')]

In [95]:

function_response_part = types.Part.from_function_response(
                name='get_city_name',
                response={'output':'Jaipur, Rajasthan, India'},
            )

function_response_content = types.Content(
            role='tool', 
            parts=[function_response_part]
        )

response = chat.send_message(
    message=function_response_part,
    config = types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=[city_function])],
    )
)

print(response.text)
print(response.function_calls)

Jaipur, Rajasthan, India

None


In [96]:
response = chat.send_message("Thanks. Tell me more about it.")
print(response.text)
print(response.function_calls)

You're welcome! Jaipur, often called the "Pink City," is the capital of Rajasthan, India. The "city with multiple windows" you mentioned likely refers to the **Hawa Mahal** (Palace of Winds). Here's why:

*   **Hawa Mahal:** This iconic landmark is a five-story structure with a facade of 953 small windows (jharokhas) decorated with intricate latticework. It was built in 1799 by Maharaja Sawai Pratap Singh so royal ladies could observe street festivals and daily life without being seen.

**Here's more about Jaipur:**

*   **History:** Jaipur was founded in 1727 by Maharaja Sawai Jai Singh II, a Kachwaha Rajput ruler. It was one of India's first planned cities.
*   **Pink City:** The city got its nickname because in 1876, it was painted pink (a color associated with hospitality) to welcome Prince Albert.
*   **Architecture:** Jaipur is known for its beautiful architecture, blending Rajput, Mughal, and European styles.
*   **Major Attractions:**

    *   **Amer Fort:** A stunning fort pal

In [98]:
chat.get_history()

[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='name of the city with multiple windows?')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'description': 'city with multiple windows'}, name='get_city_name'), function_response=None, inline_data=None, text=None)], role='model'),
 UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(id=None, name='get_city_name', response={'output': 'Jaipur, Rajasthan, India'}), inline_data=None, text=None)], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

In [101]:
response = chat.send_message(
    message="Then what is the city with blue walls",
    config = types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=[city_function])],
    )
)
print(response.text)
print(response.function_calls)

None
[FunctionCall(id=None, args={'description': 'city with blue walls'}, name='get_city_name')]


In [102]:

function_response_part = types.Part.from_function_response(
                name='get_city_name',
                response={'output':'Jodhpur, Rajasthan, India'},
            )

response = chat.send_message(
    message=function_response_part,
    config = types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=[city_function])],
    )
)

print(response.text)
print(response.function_calls)

Jodhpur, Rajasthan, India

None


In [103]:
response = chat.send_message(
    message="What is common history between these two cities?",
    config = types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=[city_function])],
    )
)
print(response.text)
print(response.function_calls)

Jaipur and Jodhpur, both located in Rajasthan, India, share several common historical threads:

*   **Rajputana Heritage:** Both cities were founded and ruled by Rajput clans. Jaipur was the capital of the Kachwaha Rajputs, while Jodhpur was the stronghold of the Rathore Rajputs. Their histories are intertwined with the broader history of Rajputana (now Rajasthan).
*   **Princely States:** During British colonial rule in India, both Jaipur and Jodhpur were princely states, meaning they maintained a degree of autonomy under British paramountcy. They had their own rulers, administrations, and armies.
*   **Strategic Importance:** Due to their location in the desert region of Rajasthan, both cities held strategic importance throughout history. They served as centers of trade, commerce, and military power.
*   **Alliance with the Mughals:** Both the Kachwaha and Rathore rulers, at various points in history, formed alliances with the Mughal Empire. These alliances influenced their cultures,